# Demo: Hello Vector World (RGB Colors)

This notebook demonstrates the basics of Vector Search using simplified RGB color vectors.
We will store 3-dimensional vectors representing colors (Red, Green, Blue) and search for the closest match to a specific color.

In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct

## 0. Connect to Qdrant

Make sure you have a Qdrant instance running locally. You can start one using Docker:

```bash
docker run -p 6333:6333 qdrant/qdrant
```

In [ ]:
client = QdrantClient(url="http://localhost:6333")

## 1. Create Collection

We will create a collection named `rgb_colors`.
The vectors will have a size of **3**, corresponding to the Red, Green, and Blue channels.
We'll use **Cosine** distance to measure similarity.

In [ ]:
collection_name = "rgb_colors"

if client.collection_exists(collection_name):
    client.delete_collection(collection_name)

client.create_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=3, distance=Distance.COSINE),
)

print(f"Created collection: {collection_name}")

## 2. Upsert (Insert) Data

We will insert 4 colors into our collection:
- **Red**
- **Green**
- **Blue**
- **Yellow** (which is a mix of Red and Green)

Each point has an ID, a vector, and a payload (metadata) with the color name.

In [ ]:
operation_info = client.upsert(
    collection_name=collection_name,
    wait=True,
    points=[
        PointStruct(id=1, vector=[1.0, 0.05, 0.05], payload={"color": "Red"}),
        PointStruct(id=2, vector=[0.05, 1.0, 0.05], payload={"color": "Green"}), 
        PointStruct(id=3, vector=[0.05, 0.05, 1.0], payload={"color": "Blue"}),
        PointStruct(id=4, vector=[1.0, 1.0, 0.05],  payload={"color": "Yellow"}),
    ],
)

print("Upserted 4 colors: Red, Green, Blue, Yellow.")

## 3. Search

Now let's search! We have a mystery color that is mostly Red, but a little bit Green.
Vector: `[1.0, 0.2, 0.0]` (Dark Orange-ish)

We expect the closest match to be **Red**.

In [ ]:
search_vector = [1.0, 0.2, 0.0]
print(f"Searching for vector {search_vector} (Reddish)...")

results = client.query_points(
    collection_name=collection_name,
    query=search_vector,
    limit=3
).points

### Results

In [ ]:
print("Results:")
for hit in results:
    print(f" - Found: {hit.payload['color']} (Score: {hit.score:.4f})")